# Basic

> Introduction to the basic functionalities & properties of FastHTML

In [ ]:
#| default_exp basic

In [ ]:
#| hide
from nbdev.showdoc import show_doc
from nbdev import nbdev_export

## Initialization

Let's create a fastHTML app with just one endpoint and run it on separate thread spawn from Jupyter Notebook.

In [3]:
from fasthtml.common import *
from fasthtml.jupyter import *

In [68]:
app = FastHTML()

In [41]:
@app.get("/")  # Define API endpoint
def index():
    # Return the HTML structure to be rendered
    return Titled('Hello, Jupyter',
        P('Welcome to the FastHTML + Jupyter example')
    )

In [70]:
# Server for Jupyter notebook web-app
# This will spawn a separate thread for running the server on default port 8000
server = JupyUvi(app)

By default, the server is run on port 8000. Once that port is occupied, the server cannot run in it and will prompt an error.  

After started, the server can be halted via the command:

In [91]:
server.stop()

Timeout


Similarly, the server can also be restarted:

In [24]:
server.start()

Note that a server created this way is tied to the initiated `app` object. It is also automatically updated. This means that whenever an API endpoint is added or modified, the changed can be immediately applied for the next user-triggered rendering or reloading.

In [74]:
#| hide
def reset_app():
    global app
    global server
    
    server.stop()
    app = FastHTML()
    server = JupyUvi(app)

## Custom HTML structure

fastHTML allows complete control over the structure of your rendered UI. Try adding some new nested elements into the main page:

In [43]:
@app.get("/")  # Define API endpoint
def index():
    # Return the HTML structure to be rendered
    return Titled('Hello, Jupyter',
        P('Welcome to the FastHTML + Jupyter example'),
        Ul(
            Li('It is easy to create.'),
            Li('It is fast to render.'),
            Li('It is fun!')
        )
    )

After reloading the webpage, the generated UI should be consistent with the description via fastHTML Python SDK:

![List rendering](images/basic_list.png)

Apart from the structure, you can also configure the attributes of your components, including their IDs, classes, or styles:

In [44]:
@app.get("/")  # Define API endpoint
def index():
    # Return the HTML structure to be rendered
    return Titled('Hello, Jupyter',
        P('Welcome to the FastHTML + Jupyter example', style="background-color: #800000;"),
        Ul(
            Li('It is easy to create.'),
            Li('It is fast to render.'),
            Li('It is fun!'),
            id="benefits"
        )
    )

## API - driven

The selection and content of rendered UI in fastHTML is dependent on API endpoint. For instance, we can define a new page for our website by adding it at endpoint GET /hello. 

In [47]:
@app.get("/welcome")
def say_hello():
    return Titled('Introduction',
        P('Welcome!'),
        P('This is an application created with FastHTML and Jupyter Notebook')
    )

As such, our application is now a multi-page application. We can also add a navigation link between the websites:

In [49]:
@app.get("/")  # Define API endpoint
def index():
    # Return the HTML structure to be rendered
    return Titled('Hello, Jupyter',
        P('Welcome to the FastHTML + Jupyter example', style="background-color: #800000;"),
        A("Welcome page ->", href="/welcome"),
        Ul(
            Li('It is easy to create.'),
            Li('It is fast to render.'),
            Li('It is fun!'),
            id="benefits"
        )
    )

@app.get("/welcome")
def say_hello():
    return Titled('Introduction',
        P('Welcome!'),
        P('This is an application created with FastHTML and Jupyter Notebook'),
        A("Home page ->", href="/")
    )

Furthermore, we can create dynamic page from path parameters or query parameters:

In [50]:
@app.get("/welcome/{name}")
def say_hello(name: str = ""):
    return Titled('Introduction',
        P(f'Welcome {name}!'),
        P('This is an application created with FastHTML and Jupyter Notebook'),
        A("Home page ->", href="/")
    )

This also applies to other inputs and plug-ins for API endpoint, such as middleware, headers, ... These will be automatically checked to extract an input with the same specified name and type.

## User interactions

As an API-driven structure, each user interaction can be handled by calling an API endpoint. The result of this responding endpoint will be rendered on browser. In fastHTML, such process can result in a full-page re-rendering based on the updated page, or a partial re-rendering by utilizing HTMX.

In [69]:
benefits = [
    'It is easy to create.',
    'It is fast to render.',
    'It is fun!'
]

@app.get("/")  # Define API endpoint
def index():
    # Return the HTML structure to be rendered
    return Titled('Hello, Jupyter',
        P('Welcome to the FastHTML + Jupyter example', style="background-color: #800000;"),
        A("Welcome page ->", href="/welcome"),
        Ul(
            # Create a list element for each benefit
            *[Li(benefit) for benefit in benefits],
            id="benefits"
        ),
        A("Benefit Form ->", href="/form")
    )

@app.get("/form")
def open_form():
    return Main(P("Add a benefit with the form below:"),
                Form(Input(type="text", name="benefit"),
                     Button("Submit"),
                     action="/form", method="post"))

@app.post("/form")
def submit_form(benefit:str):  # `benefit` correspond with `name` for the input in form
    benefits.append(benefit)
    return index()

In this case, the list of benefits is updated using a global variable on the server (not recommended unless using a local web-application). The button triggers the POST /form endpoint which adds the new benefit and re-render the whole main page.

However, it is enough in this use case to just render one element - the benefit list. This will result in higher rendering speed and yield more efficient performance. Such partial rendering can be achieved in fastHTML using HTMX. To utilize HTMX rendering, one should set:

- Function or endpoint: The function to execute when the event is triggered. This should be set using different `hx` action parameters for different methods. In this case, we can use `hx_post` and set it to be `hx_post=submit_form` or `hx_post='/form'.`  
- Target: The element to be modified after this event. In that case, it is the benefit list, so we should set `hx_target='#benefits'`.  
- Change location: How to change the content of target. In this case, we need to add one new element at the end of the list, so we can set `hx_swap='beforeend'`.

In [88]:
#| hide
reset_app()

In [89]:
benefits = [
    'It is easy to create.',
    'It is fast to render.',
    'It is fun!'
]

@app.get("/")  # Define API endpoint
def index():
    # Return the HTML structure to be rendered
    return Titled('Hello, Jupyter',
        P('Welcome to the FastHTML + Jupyter example', style="background-color: #800000;"),
        A("Welcome page ->", href="/welcome"),
        Ul(
            # Create a list element for each benefit
            *[Li(benefit) for benefit in benefits],
            id="benefits"
        ),
        P("Add a benefit with the form below:"),
        Form(hx_post='/form', hx_target='#benefits', hx_swap='beforeend')(
            Input(type="text", name="benefit"),
            Button("Submit"),
        )
    )

@app.post("/form")
def submit_form(benefit:str):  # `benefit` correspond with `name` for the input in form
    return Li(benefit)

In [55]:
#| hide
nbdev_export()